In [4]:

import pandas as pd
import nltk
import datetime
import sys
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import wordpunct_tokenize
from nltk.chunk import ne_chunk
from nltk.corpus import treebank_chunk
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import defaultdict
from operator import itemgetter
from nltk.stem.snowball import SnowballStemmer
from __future__ import division
%matplotlib inline  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
import math
from sklearn.feature_extraction.text import TfidfTransformer

from collections import defaultdict

from gensim import corpora, models, similarities

from __future__ import division

import pandas as pd
import pyLDAvis
import graphlab as gl
import pyLDAvis.graphlab
import pyLDAvis.gensim as gensimvis
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [5]:
##Import and clean data

In [435]:
url='<option value=*>'


In [439]:
urls[0]

'<option value=1984>'

In [436]:
urls=[]

In [438]:
for i in spark_title:
    urls.append(url.replace('*',i))

In [431]:
q=pd.DataFrame()

In [432]:
q['d']=urls

In [433]:
q.to_csv('names.csv')

In [447]:
book_spark=pd.read_csv('spark.csv',index_col=0)

In [448]:
book_spark['spark_title']=book_spark['spark_title'].astype(str)

In [7]:
spark_title=list(book_spark['spark_title'])

In [8]:
spark_syn=list(book_spark['spark_summary'])

In [450]:
book_spark.to_csv('sparks.csv')

In [491]:
#book_spark

In [9]:
stop_books = stopwords.words('english')
#stop_books+=stopwords.words('spanish')
stop_books += ['.', ',', '(', ')', "'", '"',u'novel',u'books',
               u'published',u'characters',u'works',u'mr.',u'ms.'
              ,u"'s",u"time",u"only",u"n't",u"book",u'life',u'one',
          u'story',u'one',u'story',u'tells',u'things','..',"&","must",'new','like',"she's","he's","gets","-","--","/",":","shes","hes",
              "get","like","--the",'"the','says','two','him.']

In [10]:
texts=[[word for word in document.lower().split() 
        if word not in stop_books] for document in spark_syn]

In [11]:
frequency=defaultdict(int)

In [12]:
for text in texts:
    for token in text:
        frequency[token] += 1

In [13]:
texts=[[token for token in text if frequency[token]>1] for text in texts]

In [14]:
dictionary = corpora.Dictionary(texts)

In [15]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [125]:
#Models

In [16]:
lsi = models.LsiModel(corpus, id2word=dictionary,num_topics=14) #num_topics=


In [17]:
tfidf=models.TfidfModel(corpus,id2word=dictionary)


In [18]:
lda = models.LdaModel(corpus,id2word=dictionary,passes=5,num_topics=4) #num_topics=

%time

[WARNING] too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [240]:
#Search/comparison

In [598]:
spark_title[196]

'The Great Gatsby'

In [599]:
spark_syn[196]

'Nick Carraway, a young man from Minnesota, moves to New York in the summer of 1922 to learn about the bond business. He rents a house in the West Egg district of Long Island, a wealthy but unfashionable area populated by the new rich, a group who have made their fortunes too recently to have established social connections and who are prone to garish displays of wealth. Nick s next-door neighbor in West Egg is a mysterious man named Jay Gatsby, who lives in a gigantic Gothic mansion and throws extravagant parties every Saturday night.,Nick is unlike the other inhabitants of West Egg he was educated at Yale and has social connections in East Egg, a fashionable area of Long Island home to the established upper class. Nick drives out to East Egg one evening for dinner with his cousin, Daisy Buchanan, and her husband, Tom, an erstwhile classmate of Nick s at Yale. Daisy and Tom introduce Nick to Jordan Baker, a beautiful, cynical young woman with whom Nick begins a romantic relationship. N

In [611]:
query=spark_syn[196]

In [612]:
search = query

In [613]:
search_vec = dictionary.doc2bow(search.lower().split())

In [614]:
#lsi vector search,index
vec_lsi = lsi[search_vec]
index_lsi = similarities.MatrixSimilarity(lsi[corpus])
sims_lsi = index_lsi[vec_lsi]

[WARNING] scanning corpus to determine the number of features (consider setting `num_features` explicitly)


In [615]:
#tfidf vector,search and index
vec_tfidf=tfidf[search_vec]
index_tfidf = similarities.MatrixSimilarity(tfidf[corpus])
sims_tfidf=index_tfidf[vec_tfidf]

[WARNING] scanning corpus to determine the number of features (consider setting `num_features` explicitly)


In [616]:
#lda model and vector
vec_lda=lda[search_vec]
index_lda=similarities.MatrixSimilarity(lda[corpus])
sims_lda=index_lda[vec_lda]


[WARNING] scanning corpus to determine the number of features (consider setting `num_features` explicitly)


In [617]:
#index and search regular model
index = similarities.MatrixSimilarity(corpus)
sims=index[search_vec]

[WARNING] scanning corpus to determine the number of features (consider setting `num_features` explicitly)


In [618]:
#get top results

In [619]:
lda_top = sorted(enumerate(sims_lda), key=lambda item: -item[1])
lsi_top = sorted(enumerate(sims_lsi), key=lambda item: -item[1])
tfidf_top = sorted(enumerate(sims_tfidf), key=lambda item: -item[1])
bag_top = sorted(enumerate(sims), key=lambda item: -item[1])

In [620]:
combined=(sims_lsi*5+sims_lda*.5+sims_tfidf)/2
comb_top= sorted(enumerate(combined), key=lambda item: -item[1])

In [621]:
for i in range(0,10):
    print (spark_title[comb_top[i][0]],comb_top[i][1],comb_top[i][0])


('The Great Gatsby', 3.25, 196)
("Who's Afraid of Virginia Woolf?", 2.6949365, 547)
('The Grapes of Wrath', 2.6944606, 194)
('The Glass Menagerie', 2.6859775, 185)
('The Adventures of Tom Sawyer', 2.6859412, 7)
('The Hours', 2.6734965, 229)
('Native Son', 2.6611631, 358)
('When the Legends Die', 2.6544785, 542)
('Lolita', 2.6443386, 298)
('A Streetcar Named Desire', 2.642144, 483)


In [625]:
model_top=tfidf_top
for i in range(0,10):
    print (spark_title[model_top[i][0]],model_top[i][1],model_top[i][0])


('The Great Gatsby', 1.0, 196)
('Daisy Miller', 0.18214376, 122)
('When the Legends Die', 0.16498812, 542)
('In Our Time', 0.13086538, 253)
('Tom Jones', 0.10730635, 508)
('Uncle Tom\xe2\x80\x99s Cabin', 0.10617765, 525)
('The Adventures of Tom Sawyer', 0.1004753, 7)
('For Whom the Bell Tolls', 0.091569677, 172)
('The Grapes of Wrath', 0.088610545, 194)
("Who's Afraid of Virginia Woolf?", 0.069454931, 547)


In [492]:
#sims_lsi

In [36]:
df_rec=pd.DataFrame()

In [37]:
df_rec['lda']=sims_lda
df_rec['lsi']=sims_lsi
df_rec['tfidf']=sims_tfidf


In [38]:
#df_rec=df_rec.drop(df_rec.index[204])

In [39]:
df_norm = (df_rec - df_rec.mean()) / (df_rec.max() - df_rec.min())

In [40]:
df_norm['total']=df_norm['lda']+df_norm['lsi']+df_norm['tfidf']

In [41]:
df_norm['title']=spark_title

In [42]:
df_norm=df_norm.sort_values(by="total",ascending=False)

In [519]:
dd=list(df_norm['title'])

In [520]:
dd[0]

'A Clockwork Orange'

In [43]:
df_norm.head()

,lda,lsi,tfidf,total,title
101,0.688148,0.191981,0.993274,1.873403,A Clockwork Orange
377,0.688148,0.177333,-0.002967,0.862514,Of Mice and Men
413,0.688148,0.169396,-0.000022,0.857522,The Power and the Glory
438,0.688148,0.160163,0.005968,0.854279,Rita Hayworth and the Shawshank Redemption
383,0.688148,0.161798,0.003054,0.853000,One Day in the Life of Ivan Denisovich


In [635]:
lda = models.LdaModel(corpus,id2word=dictionary,passes=10,num_topics=4) #num_topics=


In [879]:
pyLDAvis.save_html(vis_data,'cliff_lda.html')

In [634]:
pyLDAvis.prepared_data_to_html(vis_data)

u'\n<link rel="stylesheet" type="text/css" href="https://cdn.rawgit.com/bmabey/pyLDAvis/files/ldavis.v1.0.0.css">\n\n\n<div id="ldavis_el239247146747682409335067"></div>\n<script type="text/javascript">\n\nvar ldavis_el239247146747682409335067_data = {"plot.opts": {"xlab": "PC1", "ylab": "PC2"}, "topic.order": [1, 5, 2, 4, 3], "token.table": {"Topic": [1, 2, 4, 5, 1, 3, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 2, 3, 1, 1, 2, 3, 4, 5, 1, 3, 5, 1, 2, 3, 4, 5, 3, 5, 2, 3, 5, 5, 5, 3, 1, 3, 4, 5, 3, 1, 2, 3, 4, 5, 4, 3, 1, 2, 3, 4, 5, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 5, 1, 1, 1, 2, 4, 1, 2, 1, 2, 3, 1, 1, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 1, 3, 5, 5, 5, 4, 5, 1, 3, 4, 5, 1, 1, 1, 4, 4, 4, 4, 3, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 3, 4, 5, 4, 4, 1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 2, 3, 5, 3, 5, 1, 2, 3, 4, 5, 5, 4, 1, 4, 2, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 3, 4, 2, 5, 2, 2, 2, 5, 5, 2, 5, 1, 2, 3, 4, 1, 2, 4, 5, 1, 5, 3, 4, 3, 5, 2, 1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 2, 1, 2, 3, 4, 

In [636]:
pyLDAvis.enable_notebook()
vis_data = gensimvis.prepare(lda, corpus, dictionary)


In [637]:
pyLDAvis.display(vis_data)


### K-Means Cluster

In [647]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [648]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in spark_syn:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [59]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame'

there are 531941 items in vocab_frame


In [649]:
k_stop = stopwords.words('english')
k_stop += ['.', ',', '(', ')', "'", '"',u'novel',u'books',
               u'published',u'characters',u'works',u'mr.',u'ms.'
              ,u"'s",u"time",u"only",u"n't",u"book",u'life',u'one',
          u'story',u'one',u'story',u'tells',u'things','']

In [650]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.4,
                                 min_df=0.05, stop_words=k_stop,
                                 use_idf=True, tokenizer=tokenize_and_stem)

%time tfidf_matrix = tfidf_vectorizer.fit_transform(spark_syn) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 27.5 s, sys: 373 ms, total: 27.9 s
Wall time: 27.9 s
(562, 1156)


In [65]:
terms = tfidf_vectorizer.get_feature_names()

In [66]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)


In [165]:
from sklearn import cluster

In [96]:
dbscan = cluster.DBSCAN(eps=.8)

In [97]:
dbscan.fit(tfidf_matrix)

DBSCAN(algorithm='auto', eps=0.8, leaf_size=30, metric='euclidean',
    min_samples=5, p=None, random_state=None)

In [98]:
clusters=dbscan.labels_.tolist()

In [99]:
films = { 'title': spark_title,'synopsis': spark_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster', 'genre'])

In [100]:
frame['cluster'].value_counts()

-1    552
 0     10
Name: cluster, dtype: int64

In [ ]:
#affinity propagation

In [411]:
affinity_propagation = cluster.AffinityPropagation(damping=.9)

In [412]:
%time affinity_propagation.fit(tfidf_matrix)

CPU times: user 818 ms, sys: 8.56 ms, total: 826 ms
Wall time: 826 ms


AffinityPropagation(affinity='euclidean', convergence_iter=15, copy=True,
          damping=0.9, max_iter=200, preference=None, verbose=False)

In [413]:
clusters=affinity_propagation.labels_.tolist()

In [414]:
films = { 'title': spark_title,'synopsis': spark_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster'])

In [415]:
frame['cluster'].max()

67

In [680]:
#frame['cluster'].value_counts()

In [ ]:
#kmeans

In [874]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 448 ms, sys: 3.92 ms, total: 452 ms
Wall time: 451 ms


In [875]:
films = { 'title': spark_title,'synopsis': spark_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster'])

In [876]:
#frame.to_csv('spark_frame.csv')

In [877]:
frame['cluster'].value_counts()

0    326
1    133
2     70
3     19
4     14
Name: cluster, dtype: int64

In [873]:
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
clusterd=[]
for i in range(num_clusters):    
    yo=[]
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        yo.append(vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'))    
    clusterd.append(yo)

In [804]:
clusterd

[['mrs.', 'miss', 'london', 'lady', 'marriage', 'husband'],
 ['narrated', 'town', 'wife', 'woman', 'war', 'daughter'],
 ['king', 'henry', 'prince', 'army', 'ring', 'god'],
 ['tom', 'summer', 'slaves', 'mrs.', 'river', 'town'],
 ['boy', 'school', 'white', 'black', 'john', 'girl']]

In [805]:
spark_cluster=pd.DataFrame()

In [806]:
spark_cluster['cluster']=[0,1,2,3,4]
spark_cluster['words']=clusterd


In [675]:
spark_cluster

,cluster,words
0,0,"[enter, ring, king, murderer, doors, appeared]"
1,1,"[mrs., miss, lady, london, marriage, aunt]"
2,2,"[tom, ship, camp, john, war, army]"
3,3,"[henry, king, lords, york, london, battle]"
4,4,"[narrated, school, town, girl, children, woman]"


In [677]:
frame=frame.merge(spark_cluster,on='cluster')

In [679]:
frame.head()

,title,cluster,words
0,1984,4,"[narrated, school, town, girl, children, woman]"
1,A&P,4,"[narrated, school, town, girl, children, woman]"
2,The Age of Innocence,4,"[narrated, school, town, girl, children, woman]"
3,All the King's Men,4,"[narrated, school, town, girl, children, woman]"
4,Angela's Ashes,4,"[narrated, school, town, girl, children, woman]"


In [678]:
frame.to_csv('spark_frame.csv')

In [746]:
print("Top terms per cluster:")
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print "Cluster %d words:" % i
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print ' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore')

Top terms per cluster:
Cluster 0 words:
 henry
 king
 york
 london
 lords
 battle
Cluster 1 words:
 tom
 boy
 school
 black
 white
 girl
Cluster 2 words:
 john
 women
 daughter
 village
 mary
 narrated
Cluster 3 words:
 narrated
 wife
 town
 letter
 murderer
 woman
Cluster 4 words:
 king
 ship
 army
 ring
 god
 captain
Cluster 5 words:
 mrs.
 miss
 lady
 marriage
 london
 aunt


In [490]:

from os import path
from wordcloud import WordCloud


# Read the whole text.
text = spark_syn

# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")

# take relative word frequencies into account, lower max_font_size
wordcloud = WordCloud(max_font_size=40, relative_scaling=.5).generate(text)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

# The pil way (if you don't have matplotlib)
#image = wordcloud.to_image()
#image.show()

TypeError: expected string or buffer

In [ ]:
#ward hierarchal clustering

In [170]:
from sklearn.neighbors import kneighbors_graph


In [313]:
connectivity = kneighbors_graph(tfidf_matrix, n_neighbors=5, include_self=False)

In [306]:
ward = cluster.AgglomerativeClustering(n_clusters=5, linkage='ward',connectivity=connectivity)

In [307]:
y=tfidf_matrix.todense()

In [275]:
ward.fit(y)


AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=<562x562 sparse matrix of type '<type 'numpy.float64'>'
	with 2810 stored elements in Compressed Sparse Row format>,
            linkage='ward', memory=Memory(cachedir=None), n_clusters=5,
            n_components=None, pooling_func=<function mean at 0x106752410>)

In [276]:
clusters = ward.labels_.tolist()

In [277]:
films = { 'title': spark_title,'synopsis': spark_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster'])

In [278]:
frame['cluster'].value_counts()

1    492
0     27
3     24
2     10
4      9
Name: cluster, dtype: int64

In [279]:
#

,title,cluster
1,1984,1
1,2001: A Space Odyssey,1
1,A&P,1
0,"Absalom, Absalom!",0
1,Across Five Aprils,1


In [ ]:
#cosine linkage

In [308]:
agglo_cluster = cluster.AgglomerativeClustering(linkage="average", affinity="cosine", n_clusters=4,connectivity=connectivity)

In [309]:
agglo_cluster.fit(y)

AgglomerativeClustering(affinity='cosine', compute_full_tree='auto',
            connectivity=<562x562 sparse matrix of type '<type 'numpy.float64'>'
	with 1124 stored elements in Compressed Sparse Row format>,
            linkage='average', memory=Memory(cachedir=None), n_clusters=4,
            n_components=None, pooling_func=<function mean at 0x106752410>)

In [310]:
clusters = agglo_cluster.labels_.tolist()

In [311]:
films = { 'title': spark_title,'synopsis': spark_syn, 'cluster': clusters }

frame = pd.DataFrame(films, index = [clusters] , columns = [ 'title', 'cluster'])

In [312]:
frame['cluster'].value_counts()

0    559
3      1
2      1
1      1
Name: cluster, dtype: int64

In [457]:
from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
from scipy.cluster.vq import kmeans,vq

[INFO] GraphLab Create v1.8.3 started. Logging: /tmp/graphlab_server_1456968691.log
